In [13]:
def read_all_text(file):
    with open(file) as f:
        return f.read()

In [14]:
text = read_all_text("shakespeare.txt")

In [15]:
text

"THE SONNETS\n\nby William Shakespeare\n\nFrom fairest creatures we desire increase,\nThat thereby beauty's rose might never die,\nBut as the riper should by time decease,\nHis tender heir might bear his memory:\nBut thou contracted to thine own bright eyes,\nFeed'st thy light's flame with self-substantial fuel,\nMaking a famine where abundance lies,\nThy self thy foe, to thy sweet self too cruel:\nThou that art now the world's fresh ornament,\nAnd only herald to the gaudy spring,\nWithin thine own bud buriest thy content,\nAnd tender churl mak'st waste in niggarding:\nPity the world, or else this glutton be,\nTo eat the world's due, by the grave and thee.\n\nWhen forty winters shall besiege thy brow,\nAnd dig deep trenches in thy beauty's field,\nThy youth's proud livery so gazed on now,\nWill be a tattered weed of small worth held:  \nThen being asked, where all thy beauty lies,\nWhere all the treasure of thy lusty days;\nTo say within thine own deep sunken eyes,\nWere an all-eating 

In [19]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("".join(chars))
print(vocab_size)


 !'(),-.:;?ABCDEFGHIJKLMNOPRSTUVWYabcdefghijklmnopqrstuvwxyz
61


In [20]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encode a string into a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decode a list of integers into a string

In [23]:
print(encode("Hello world!"))
print(decode(encode("Hello world!")))

[19, 39, 46, 46, 49, 1, 57, 49, 52, 46, 38, 2]
Hello world!


In [28]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:1000])

torch.Size([94275])
tensor([30, 19, 16,  1, 29, 26, 25, 25, 16, 30, 29,  0,  0, 36, 59,  1, 33, 43,
        46, 46, 43, 35, 47,  1, 29, 42, 35, 45, 39, 53, 50, 39, 35, 52, 39,  0,
         0, 17, 52, 49, 47,  1, 40, 35, 43, 52, 39, 53, 54,  1, 37, 52, 39, 35,
        54, 55, 52, 39, 53,  1, 57, 39,  1, 38, 39, 53, 43, 52, 39,  1, 43, 48,
        37, 52, 39, 35, 53, 39,  6,  0, 30, 42, 35, 54,  1, 54, 42, 39, 52, 39,
        36, 59,  1, 36, 39, 35, 55, 54, 59,  3, 53,  1, 52, 49, 53, 39,  1, 47,
        43, 41, 42, 54,  1, 48, 39, 56, 39, 52,  1, 38, 43, 39,  6,  0, 13, 55,
        54,  1, 35, 53,  1, 54, 42, 39,  1, 52, 43, 50, 39, 52,  1, 53, 42, 49,
        55, 46, 38,  1, 36, 59,  1, 54, 43, 47, 39,  1, 38, 39, 37, 39, 35, 53,
        39,  6,  0, 19, 43, 53,  1, 54, 39, 48, 38, 39, 52,  1, 42, 39, 43, 52,
         1, 47, 43, 41, 42, 54,  1, 36, 39, 35, 52,  1, 42, 43, 53,  1, 47, 39,
        47, 49, 52, 59,  9,  0, 13, 55, 54,  1, 54, 42, 49, 55,  1, 37, 49, 48,
        54, 52, 35, 

In [29]:
n = int(0.9 * len(data))
train_data = data[:n] # first 90% will be train, rest val
val_data = data[n:]

In [30]:
block_size = 8
train_data[:block_size+1]

tensor([30, 19, 16,  1, 29, 26, 25, 25, 16])

In [31]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([30]) the target: 19
when input is tensor([30, 19]) the target: 16
when input is tensor([30, 19, 16]) the target: 1
when input is tensor([30, 19, 16,  1]) the target: 29
when input is tensor([30, 19, 16,  1, 29]) the target: 26
when input is tensor([30, 19, 16,  1, 29, 26]) the target: 25
when input is tensor([30, 19, 16,  1, 29, 26, 25]) the target: 25
when input is tensor([30, 19, 16,  1, 29, 26, 25, 25]) the target: 16


In [33]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences we process in parallel
block_size = 8 # maximum context length for predictions

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("---")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input it {context.tolist()} then target: {target}")

inputs:
torch.Size([4, 8])
tensor([[54, 42, 59,  1, 46, 49, 56, 39],
        [42,  1, 54, 42, 39,  1, 38, 52],
        [ 1, 53, 35, 47, 39,  1, 41, 52],
        [52, 49, 55, 41, 42, 54,  0, 30]])
targets:
torch.Size([4, 8])
tensor([[42, 59,  1, 46, 49, 56, 39,  6],
        [ 1, 54, 42, 39,  1, 38, 52, 49],
        [53, 35, 47, 39,  1, 41, 52, 49],
        [49, 55, 41, 42, 54,  0, 30, 49]])
---
when input it [54] then target: 42
when input it [54, 42] then target: 59
when input it [54, 42, 59] then target: 1
when input it [54, 42, 59, 1] then target: 46
when input it [54, 42, 59, 1, 46] then target: 49
when input it [54, 42, 59, 1, 46, 49] then target: 56
when input it [54, 42, 59, 1, 46, 49, 56] then target: 39
when input it [54, 42, 59, 1, 46, 49, 56, 39] then target: 6
when input it [42] then target: 1
when input it [42, 1] then target: 54
when input it [42, 1, 54] then target: 42
when input it [42, 1, 54, 42] then target: 39
when input it [42, 1, 54, 42, 39] then target: 1
when inpu

In [48]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 61])
tensor(4.3165, grad_fn=<NllLossBackward0>)

PkW
DPi,ozKIgI
?YnTs,,HI(fjt HbzpTEIqA(!k!mio:q-F-hHCD: nAti(mIc.qAwjV,u
s,uHd,o,uC nnn!-;HG).BMSE E


In [49]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [67]:
batch_size = 32

for steps in range(10000):
    xb, yb = get_batch("train")

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4145429134368896


In [68]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=400)[0].tolist()))


Tatheserie.


Whak sue tundutw peplece bet beay myost Inavewinkedspu,
No l,
Winexe'senstthursegichys wazeane schou mu id heme, me bene th'suryes arstearer ouew, whon stetr f din se t,
Sune trillo wove h, o wt theiowate w is the,
Nofr jugu mby celike Feathye 
Yosuer wof s t oont) wndossuee s,
Ayothenthanere,

I clle nd ble!
Thoney'sof se hedom foudelewangeng hat our m I she oreaveg ays oung nd te,



In [69]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [70]:
# The mathematical trick in self-attention

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch,time,channel
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [73]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        # averaging tokens like this results in a lot of loss of information, such as sequential information (where are the characters related to each other)
        xbow[b, t] = torch.mean(xprev, 0)

In [72]:
xbow

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [75]:
a = torch.ones(3,3)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
c

tensor([[17., 12.],
        [17., 12.],
        [17., 12.]])

In [76]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [79]:
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
print(a)
print(b)
c = a @ b
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[4., 1.],
        [6., 8.],
        [6., 9.]])
tensor([[4.0000, 1.0000],
        [5.0000, 4.5000],
        [5.3333, 6.0000]])


In [81]:
# vectorize the averaging operations

wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (impl. B, T, T) @ (B, T, C) --> ()

In [83]:
torch.allclose(xbow, xbow2)

True

In [85]:
print(xbow[0])
print(xbow2[0])

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [86]:
# even better

In [87]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x

torch.allclose(xbow, xbow3)

True

In [91]:
tril = torch.tril(torch.ones(T, T))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [92]:
wei = torch.zeros((T, T))
wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [97]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [98]:
F.softmax(wei, dim=-1)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [100]:
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
out = wei @ x

out.shape

torch.Size([4, 8, 32])

In [101]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [102]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [105]:
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
out = wei @ x

out.shape

torch.Size([4, 8, 32])

In [107]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [114]:
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
# out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [115]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)